cell 1

In [1]:


!pip uninstall -y langchain-core langgraph langgraph-checkpoint langgraph-prebuilt transformers datasets peft accelerate


!pip install -q \
    transformers==4.41.1 \
    datasets==2.19.0 \
    peft==0.11.1 \
    accelerate==0.29.3 \
    langgraph==0.0.33 \
    scikit-learn==1.5.0 \
    huggingface_hub==0.23.1 \
    numpy==1.26.4 \
    packaging==23.2 \
    fsspec==2024.3.1 \
    requests==2.32.3 \
    torch==2.7.1

print("\nKey package versions:")
!pip list | grep -E "transformers|datasets|peft|accelerate|langgraph|torch"

Found existing installation: langchain-core 0.1.53
Uninstalling langchain-core-0.1.53:
  Successfully uninstalled langchain-core-0.1.53
Found existing installation: langgraph 0.0.33
Uninstalling langgraph-0.0.33:
  Successfully uninstalled langgraph-0.0.33
Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Successfully uninstalled transformers-4.41.1
Found existing installation: datasets 2.19.0
Uninstalling datasets-2.19.0:
  Successfully uninstalled datasets-2.19.0
Found existing installation: peft 0.11.1
Uninstalling peft-0.11.1:
  Successfully uninstalled peft-0.11.1
Found existing installation: accelerate 0.29.3
Uninstalling accelerate-0.29.3:
  Successfully uninstalled accelerate-0.29.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.3.8 requires langchain-core<1.0.0,>=0.3.51, but you have langch

cell 2

In [4]:
#Import libraries and setup with working logging
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType
)
import torch
import numpy as np
from langgraph.graph import END, StateGraph
import logging
from datetime import datetime
import os
import pandas as pd
from datasets import Dataset
import sys  # Added for stdout logging

# Create logger instance
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create file handler
log_file = "classification_log.txt"
file_handler = logging.FileHandler(log_file, mode='w')
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)

# Create console handler for real-time output
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logger.addHandler(console_handler)

# Verify GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
logger.info(f"Using device: {device}")
logger.info("Logging system initialized - this should appear in file")

INFO:root:Using device: cuda


Using device: cuda
2025-06-23 21:47:12,952 - INFO - Using device: cuda
2025-06-23 21:47:12,952 - INFO - Using device: cuda


INFO:root:Logging system initialized - this should appear in file


2025-06-23 21:47:12,955 - INFO - Logging system initialized - this should appear in file
2025-06-23 21:47:12,955 - INFO - Logging system initialized - this should appear in file


cell 3

In [6]:
#Download and prepare IMDB dataset directly
# Manual download to avoid fsspec issues
!wget -q https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

def load_imdb_data(data_type='train'):
    texts = []
    labels = []

    for label in ['pos', 'neg']:
        path = f"aclImdb/{data_type}/{label}"
        for file in os.listdir(path):
            if file.endswith('.txt'):
                with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
                    texts.append(f.read())
                labels.append(1 if label == 'pos' else 0)

    return pd.DataFrame({'text': texts, 'label': labels})
# Load and split data
train_df = load_imdb_data('train')
test_df = load_imdb_data('test')

# Create datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
dataset = {'train': train_dataset, 'test': test_dataset}

# Define label mapping
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# Preprocess function
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

# Preprocess dataset
tokenized_dataset = {}
for split in dataset:
    tokenized_dataset[split] = dataset[split].map(
        preprocess_function,
        batched=True,
        remove_columns=["text"]
    )
    tokenized_dataset[split] = tokenized_dataset[split].rename_column("label", "labels")

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

cell 4

In [7]:
#Config LoRA for efficient fine-tuning
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label=id2label,
    label2id=label2id
).to(device)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=4,
    lora_alpha=32,
    lora_dropout=0.01,
    inference_mode=False,
    target_modules=["q_lin", "v_lin"]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 665,858 || all params: 67,620,868 || trainable%: 0.9847


cell 5

In [8]:
# Training setup and execution
import transformers

# Print Transformers version to confirm compatibility
print(f"Transformers version: {transformers.__version__}")

# appropriate parameter names based on version
if transformers.__version__.startswith('4.41'):
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        save_strategy="no",
        logging_dir="./logs",
        report_to="none",
        optim="adamw_torch",
        fp16=True
    )
else:
    # Fallback for newer versions
    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",  # Newer versions
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        save_strategy="no",
        logging_dir="./logs",
        report_to="none",
        optim="adamw_torch",
        fp16=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].shuffle(seed=42).select(range(1000)),
    eval_dataset=tokenized_dataset["test"].shuffle(seed=42).select(range(200)),
    compute_metrics=lambda p: {"accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()}
)

print("Starting training...")
trainer.train()
print("Training completed!")

# Save model
model.save_pretrained("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")

Transformers version: 4.41.1
Starting training...


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:469: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.680347,0.585000
2,No log,0.675574,0.615000


Training completed!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('fine-tuned-model/tokenizer_config.json',
 'fine-tuned-model/special_tokens_map.json',
 'fine-tuned-model/vocab.txt',
 'fine-tuned-model/added_tokens.json')

cell 6

In [9]:
# inference function with confidence
def classify_text(text, model, tokenizer, device):
    # Set model to evaluation mode
    model.eval()

    # Tokenize input
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256,
        padding="max_length"
    ).to(device)
    # Run inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    confidence, predicted = torch.max(probs, dim=1)

    # Return label and confidence
    return model.config.id2label[predicted.item()], confidence.item()

cell 7

In [10]:
# LangGraph state and nodes with dictionary-based state
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

# state structure
class State(TypedDict):
    text: str
    label: str
    confidence: float
    corrected_label: str
    needs_correction: bool
    user_feedback: str
def inference_node(state: State) -> dict:
    logger.info(f"Input text: {state['text']}")
    label, confidence = classify_text(state['text'], model, tokenizer, device)
    logger.info(f"[InferenceNode] Predicted: {label} | Confidence: {confidence:.2%}")
    print(f"[InferenceNode] Predicted: {label} | Confidence: {confidence:.2%}")
    return {
        "label": label,
        "confidence": confidence,
        "needs_correction": False  # Reset for new input
    }
def confidence_check_node(state: State) -> dict:
    CONFIDENCE_THRESHOLD = 0.7
    needs_correction = state["confidence"] < CONFIDENCE_THRESHOLD

    if needs_correction:
        logger.info(f"[ConfidenceCheckNode] Confidence too low. Triggering fallback...")
        print(f"[ConfidenceCheckNode] Confidence too low. Triggering fallback...")

    return {"needs_correction": needs_correction}
def fallback_node(state: State) -> dict:
    print(f"\n[FallbackNode] Could you clarify? Was this review positive or negative?")
    user_input = input("User: ").strip().lower()

    # Process user clarification
    if "positive" in user_input:
        corrected_label = "POSITIVE"
    elif "negative" in user_input:
        corrected_label = "NEGATIVE"
    else:
        corrected_label = state["label"]  # Default to original prediction

    logger.info(f"[FallbackNode] User feedback: {user_input}")
    return {
        "corrected_label": corrected_label,
        "user_feedback": user_input
    }

cell 8

In [11]:
# LangGraph workflow with proper state management
from langgraph.graph import END, StateGraph

# workflow with defined state structure
workflow = StateGraph(State)

# Add nodes
workflow.add_node("inference", inference_node)
workflow.add_node("confidence_check", confidence_check_node)
workflow.add_node("fallback", fallback_node)

# Set entry point
workflow.set_entry_point("inference")

# Connect nodes
workflow.add_edge("inference", "confidence_check")
# Conditional edge based on confidence
workflow.add_conditional_edges(
    "confidence_check",
    lambda state: "fallback" if state["needs_correction"] else END,
    {"fallback": "fallback", END: END}
)

# Connect fallback to end
workflow.add_edge("fallback", END)

# Compile the workflow
app = workflow.compile()

cell 9

In [12]:
# CLI execution loop with dictionary state
def run_cli():
    print("\n" + "="*50)
    print("Text Classification System with Self-Healing")
    print("Enter 'exit' to quit")
    print("="*50)

    while True:
        text = input("\nEnter text: ").strip()
        if text.lower() == 'exit':
            break

        #initial state
        initial_state = {
            "text": text,
            "label": "",
            "confidence": 0.0,
            "corrected_label": "",
            "needs_correction": False,
            "user_feedback": ""
        }
        # Execute workflow
        result = app.invoke(initial_state)

        # Determine final label
        final_label = result["corrected_label"] if result["corrected_label"] else result["label"]
        print(f"\nFinal Label: {final_label}")

        # Log final decision
        logger.info(f"Final decision: {final_label}")
        logger.info("-"*50)

cell 10

In [13]:
#CLI
print("Launching classification system...")
run_cli()

Launching classification system...

Text Classification System with Self-Healing
Enter 'exit' to quit

Enter text: The movie was painfully slow and boring.


INFO:root:Input text: The movie was painfully slow and boring.


2025-06-23 21:59:27,885 - INFO - Input text: The movie was painfully slow and boring.
2025-06-23 21:59:27,885 - INFO - Input text: The movie was painfully slow and boring.


INFO:root:[InferenceNode] Predicted: POSITIVE | Confidence: 51.66%


2025-06-23 21:59:27,961 - INFO - [InferenceNode] Predicted: POSITIVE | Confidence: 51.66%
2025-06-23 21:59:27,961 - INFO - [InferenceNode] Predicted: POSITIVE | Confidence: 51.66%


...

2025-06-23 21:59:59,715 - INFO - [FallbackNode] User feedback: it was definitely negative.
2025-06-23 21:59:59,715 - INFO - [FallbackNode] User feedback: it was definitely negative.


INFO:root:Final decision: NEGATIVE



Final Label: NEGATIVE
2025-06-23 21:59:59,719 - INFO - Final decision: NEGATIVE
2025-06-23 21:59:59,719 - INFO - Final decision: NEGATIVE


INFO:root:--------------------------------------------------


2025-06-23 21:59:59,721 - INFO - --------------------------------------------------
2025-06-23 21:59:59,721 - INFO - --------------------------------------------------

Enter text: exit


cell 11

In [14]:
#Display log file contents with verification
print("\nLog File Contents:")
print("="*50)

# Read and print log file
try:
    with open(log_file, 'r') as f:
        content = f.read()
        if content:
            print(content)
        else:
            print("Log file is empty. Check logger configuration.")
            print("Recent log messages from handlers:")
            for handler in logger.handlers:
                if isinstance(handler, logging.FileHandler):
                    print(f"FileHandler: {handler.baseFilename}")

            # Print test log message
            logger.info("Test message written at end of execution")
            with open(log_file, 'r') as f2:
                test_content = f2.read()
                print("\nAfter writing test message:")
                print(test_content)
except Exception as e:
    print(f"Error reading log file: {e}")
    print("Current directory contents:")
    print(os.listdir('.'))


Log File Contents:
2025-06-23 21:47:12,952 - INFO - Using device: cuda
2025-06-23 21:47:12,955 - INFO - Logging system initialized - this should appear in file
2025-06-23 21:59:27,885 - INFO - Input text: The movie was painfully slow and boring.
2025-06-23 21:59:27,961 - INFO - [InferenceNode] Predicted: POSITIVE | Confidence: 51.66%
2025-06-23 21:59:27,973 - INFO - [ConfidenceCheckNode] Confidence too low. Triggering fallback...
2025-06-23 21:59:59,715 - INFO - [FallbackNode] User feedback: it was definitely negative.
2025-06-23 21:59:59,719 - INFO - Final decision: NEGATIVE
2025-06-23 21:59:59,721 - INFO - --------------------------------------------------
025-06-23 21:59:59,719 - INFO - Final decision: NEGATIVE
2025-06-23 21:59:59,721 - INFO - --------------------------------------------------



cell 12

In [19]:
# Google Drive Saver
from google.colab import drive
import os
from datetime import datetime

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
print("Google Drive mounted!")

# folder
folder_name = "ATG_Assignment_Submission"
target_path = f"/content/drive/MyDrive/{folder_name}"
os.makedirs(target_path, exist_ok=True)
print(f"Created folder: {target_path}")

# trained model
model_path = f"{target_path}/fine-tuned-model"
!cp -r fine-tuned-model "{model_path}"
print(f"Saved model to: {model_path}")

# log file
log_path = f"{target_path}/classification_log.txt"
!cp classification_log.txt "{log_path}"
print(f"Saved log to: {log_path}")

Mounted at /content/drive
Google Drive mounted!
Created folder: /content/drive/MyDrive/ATG_Assignment_Submission
Saved model to: /content/drive/MyDrive/ATG_Assignment_Submission/fine-tuned-model
Saved log to: /content/drive/MyDrive/ATG_Assignment_Submission/classification_log.txt
